In [17]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from torch.utils.data import Dataset, DataLoader, sampler
import torchvision.datasets as dsets

In [18]:
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = np.loadtxt('./Chol_X.csv', delimiter=',', dtype='float32').tolist()
        
        y = np.loadtxt('./Chol_Y.csv', delimiter=',', dtype='float32')
        self.y_data = np.reshape(y, (len(y), 1)).tolist()                
        
        self.x_train = self.x_data[:int(len(self.x_data)*0.1)]
        self.x_valid = self.x_data[int(len(self.x_data)*0.1):int(len(self.x_data)*0.2)]
        self.x_test = self.x_data[int(len(self.x_data)*0.1):int(len(self.x_data)*0.2)]

        self.y_train = self.y_data[:int(len(self.y_data)*0.1)]
        self.y_valid = self.y_data[int(len(self.y_data)*0.1):int(len(self.y_data)*0.2)]
        self.y_test = self.y_data[int(len(self.y_data)*0.1):int(len(self.y_data)*0.2)]

        print(len(self.x_data), len(self.x_train), len(self.x_valid), len(self.x_test))
        print(len(self.y_data), len(self.y_train), len(self.y_valid), len(self.y_test))
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self,client_idx):
        if client_idx == 0:
            idx = np.random.randint(low=0,high=200,size = 16)
        elif client_idx == 1:
            idx = np.random.randint(low=200,high=300,size = 16)
        elif client_idx == 2:
            idx = np.random.randint(low=300,high=400,size = 16)
        elif client_idx == 3:
            idx = np.random.randint(low=400,high=450,size = 16)
        elif client_idx == 4:
            idx = np.random.randint(low=450,high=500,size = 16)
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(7, 50)
        self.linear2 = nn.Linear(50, 30)        
        self.linear3 = nn.Linear(30, 1)       
        self.activation = nn.LeakyReLU(1e-3)
        # self.activation = nn.ReLU()
    
    def forward(self, x):
        out = self.linear1(x)        
        out = self.activation(out)
        out = self.linear2(out)
        out = self.activation(out)
        out = self.linear3(out)
        out = self.activation(out)      
        
        return out
    
class Net_Client(nn.Module):
    def __init__(self):
        super(Net_Client, self).__init__()
        self.linear1 = nn.Linear(7, 50).type(dtype)
        self.activation = nn.LeakyReLU(5e-3)
        # self.activation = nn.ReLU()
    
    def forward(self, x):
        out = self.linear1(x).type(dtype)
        out = self.activation(out).type(dtype)        
        
        return out
    
class Net_Server(nn.Module):
    def __init__(self):
        super(Net_Server, self).__init__()        
        self.linear2 = nn.Linear(50, 30).type(dtype)
        self.linear3 = nn.Linear(30, 1).type(dtype)
        self.activation = nn.LeakyReLU(5e-3)
        # self.activation = nn.ReLU()
    
    def forward(self, x):        
        out = self.linear2(x).type(dtype)
        out = self.activation(out).type(dtype)
        out = self.linear3(out).type(dtype)
        out = self.activation(out).type(dtype)
        
        return out
    
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

In [19]:
client_N = 5

epochs = 200
learning_rate = 1e-3 # 1e-5
LR_scheduler_step_size = 100
optimizer = "Adam"

global_fraction = 0.
# train_datafraction = [0.9, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
# train_datafraction = [0.6, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01]
train_datafraction = [0.2, 0.2, 0.2, 0.2, 0.2]

batch_size = 2048
train_batch_size = [(int) (batch_size / train_datafraction[len(train_datafraction)-1] * x) for x in train_datafraction]

use_CUDA = False
use_CUDA = torch.cuda.is_available()

print("\nClient Batch Size : ", batch_size)
print("Server Batch Size : ", batch_size * client_N)
print("Epochs : ", epochs)
print("Optimizer : ", optimizer, " Optimizer")
print("Learning Rate : ", learning_rate)
print("LR Scheduler Step Size : ", LR_scheduler_step_size)
print("Train Data Size : ", train_datafraction)
print("Train Data Total Fraction : ", len(train_datafraction), "개, ", sum(train_datafraction))


Client Batch Size :  2048
Server Batch Size :  10240
Epochs :  200
Optimizer :  Adam  Optimizer
Learning Rate :  0.001
LR Scheduler Step Size :  100
Train Data Size :  [0.2, 0.2, 0.2, 0.2, 0.2]
Train Data Total Fraction :  5 개,  1.0


In [20]:
device = torch.device("cpu")
dtype = torch.FloatTensor

if use_CUDA:
    print("GPU Count : ", torch.cuda.device_count())
    if(torch.cuda.device_count() > 1):
        torch.cuda.set_device()
        device = torch.cuda.current_device()
    else:
        torch.cuda.set_device(1)
        device = torch.cuda.current_device()
    print("Selected GPU # :", torch.cuda.current_device())
    print("Selected GPU Name : ", torch.cuda.get_device_name(device=device))
    dtype = torch.cuda.FloatTensor

GPU Count :  4
Selected GPU # : 0
Selected GPU Name :  Tesla V100-DGXS-32GB


In [21]:
def get_indices(dataset,class_name):
    indices = []
    for i in range(len(dataset.targets)):
        if dataset.targets[i] == class_name:
            indices.append(i)
    return indices

In [22]:
dataset = CustomDataset()

X_train = dataset.x_train
Y_train = dataset.y_train

X_test = dataset.x_test
Y_test = dataset.y_test

dataset_size = len(X_train)
indices = list(range(dataset_size))

407540 40754 40754 40754
407540 40754 40754 40754


In [23]:
random_seed = 42

np.random.seed(random_seed)
np.random.shuffle(indices)
train_datasize = [(int)(len(X_train) * x) for x in train_datafraction]
print("Each Train Loader Size : ", train_datasize)
train_dataidx = list()
for i in range(client_N):
    train_dataidx.append(indices[(int)(dataset_size * global_fraction) : (int)(dataset_size*(global_fraction+train_datafraction[i]))])
    global_fraction += train_datafraction[i]
    
print("train_dataidx : ", len(train_dataidx))

print("Each Train Loader Batch Size : ", train_batch_size)
print("Full Batch : ", sum(train_batch_size))

Each Train Loader Size :  [8150, 8150, 8150, 8150, 8150]
train_dataidx :  5
Each Train Loader Batch Size :  [2048, 2048, 2048, 2048, 2048]
Full Batch :  10240


In [24]:
class CustomTrainDataset(Dataset):
    def __init__(self):
        self.x_data = np.loadtxt('./Chol_X.csv', delimiter=',', dtype='float32').tolist()
        
        y = np.loadtxt('./Chol_Y.csv', delimiter=',', dtype='float32')
        self.y_data = np.reshape(y, (len(y), 1)).tolist()
        
        self.x_train = self.x_data[:int(len(self.x_data)*0.1)]
        
        self.y_train = self.y_data[:int(len(self.y_data)*0.1)]
        print("set train data", len(self.x_train))
        
    def __len__(self):
        return len(self.x_train)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_train[idx])
        y = torch.FloatTensor(self.y_train[idx])
        return x, y

class CustomTestDataset(Dataset):
    def __init__(self):
        self.x_data = np.loadtxt('./Chol_X.csv', delimiter=',', dtype='float32').tolist()
        
        y = np.loadtxt('./Chol_Y.csv', delimiter=',', dtype='float32')
        self.y_data = np.reshape(y, (len(y), 1)).tolist()
        
        self.x_test = self.x_data[int(len(self.x_data)*0.1):int(len(self.x_data)*0.2)]            
        self.y_test = self.y_data[int(len(self.y_data)*0.1):int(len(self.y_data)*0.2)]
        print("set test data : ", len(self.x_test))
        
    def __len__(self):
        return len(self.x_test)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_test[idx])
        y = torch.FloatTensor(self.y_test[idx])
        return x, y

In [25]:
traindata = CustomTrainDataset()
testdata = CustomTestDataset()

set train data 40754
set test data :  40754


In [26]:
train_loader = list()

for i in range(client_N):
    train_loader.append(DataLoader(
        dataset=traindata,\
        batch_size=train_batch_size[i],\
        sampler=sampler.SubsetRandomSampler(train_dataidx[i])
    ))

test_loader = DataLoader(
    dataset=testdata,\
    batch_size=2048, \
    shuffle=True
)
print("Iteration : ", train_loader[0].__len__())

Iteration :  4


In [27]:
model_Client = list()
for idx in range(client_N):
    model_Client.append(Net_Client())
model_Server = Net_Server()

In [28]:
params_list = list()
for idx in range(client_N):
    params_list.extend(list(model_Client[idx].parameters()))
params_list.extend(list(model_Server.parameters()))

if(optimizer == "Adam"):
    opt = optim.Adam(params=params_list, lr=learning_rate)
elif(optimizer == "SGD"):
    opt = optim.SGD(params=params_list, lr=learning_rate, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer=opt, step_size=LR_scheduler_step_size, gamma=0.1)

In [29]:
def test_fn():
    with torch.no_grad():
        # count_target = [0 for i in range(10)]
        # count_pred = [0 for i in range(10)]
        # total = 0
        # correct = 0
        
        for iter, data in enumerate(test_loader):            
            test_input = data[0].cuda()
            test_target = data[1].cuda()

            output = model_Client[0](test_input)
            output = model_Server(output)
            
            # _, prediction = torch.max(output.data, 1)
            """
            for idx in range(test_target.shape[0]):
                count_target[test_target[idx].data] += 1
                count_pred[prediction[idx].data] += 1

            total += test_target.size(0)
            correct += (prediction == test_target).sum().item()
            """
        # print("count_target : ", count_target)
        # print("count_pred : ", count_pred)
        # print("total : ", total)
        # print("correct : ", correct)
        # print("Accuracy : ", correct / total * 100)
        print("Predict results : ", output[0], test_target[0])

In [30]:
train_input = list()
train_target = list()
client_output = list()
pred = list()
grad_linear1_list = list()

history = list()
hist = list()
for epoch in range(epochs):
    for iter, data in enumerate(zip(*train_loader)):
        train_input.clear()
        train_target.clear()
        client_output.clear()
        pred.clear()
        grad_linear1_list.clear()
        """
        for idx in range(3):            
            print("===========")
            print(data[idx][0]) # 값 제대로 들어감
            print(data[idx][1]) # 값 제대로 들어감            
            print("===========")
        break
        """
        for train_data_iter_client in range(client_N):            
            train_input.append(data[train_data_iter_client][0].cuda())
            train_target.append(data[train_data_iter_client][1].cuda())
        
        # check over
        # print(train_input)
        # print(train_target)
        # break

        for client_idx in range(client_N):
            client_output.append(model_Client[client_idx](train_input[client_idx]))            
        
        for client_idx in range(client_N):
            pred.append(model_Server(client_output[client_idx]))

        pred_Total = torch.cat(pred, 0)        

        target = torch.cat(train_target, 0)
        
        loss_fn = RMSLELoss()
        # loss = F.mse_loss(pred_Total, target)
        loss = loss_fn(pred_Total, target)
        opt.zero_grad()
        loss.backward()
        
        history.append(loss.item())
        
        # print('iter {:4d}/{} \t Loss: {:.6f}'.format(iter, len(train_loader[0]), loss.item()))
        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(epoch, epochs, iter+1, len(train_loader[0]), loss.item()))
        
        with torch.no_grad():
            for idx in range(client_N):
                grad_linear1_list.append(model_Client[idx].linear1.weight.grad)
            grad_linear1_mean = torch.mean(torch.stack(grad_linear1_list), dim=0)
            for idx in range(client_N):
                model_Client[idx].linear1.weight.grad = grad_linear1_mean

        opt.step()
        
        with torch.no_grad():
            if iter == 0:
                print("epoch : ", epoch, ", loss : ", loss)
                if(epoch % 30 == 0):
                    test_fn()

Epoch    0/200 Batch 1/4 Cost: 2.729175
epoch :  0 , loss :  tensor(2.7292, device='cuda:0', grad_fn=<SqrtBackward>)
Predict results :  tensor([8.4546], device='cuda:0') tensor([65.], device='cuda:0')
Epoch    0/200 Batch 2/4 Cost: 2.254445
Epoch    0/200 Batch 3/4 Cost: 1.952230
Epoch    0/200 Batch 4/4 Cost: 1.726653
Epoch    1/200 Batch 1/4 Cost: 1.561481
epoch :  1 , loss :  tensor(1.5615, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch    1/200 Batch 2/4 Cost: 1.426547
Epoch    1/200 Batch 3/4 Cost: 1.298698
Epoch    1/200 Batch 4/4 Cost: 1.201682
Epoch    2/200 Batch 1/4 Cost: 1.117294
epoch :  2 , loss :  tensor(1.1173, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch    2/200 Batch 2/4 Cost: 1.028190
Epoch    2/200 Batch 3/4 Cost: 0.962268
Epoch    2/200 Batch 4/4 Cost: 0.895657
Epoch    3/200 Batch 1/4 Cost: 0.837222
epoch :  3 , loss :  tensor(0.8372, device='cuda:0', grad_fn=<SqrtBackward>)
Epoch    3/200 Batch 2/4 Cost: 0.782177
Epoch    3/200 Batch 3/4 Cost: 0.732717
Epoch   

In [31]:
import pandas as pd
aaa = []
aaa.append(pred_Total.detach().cpu().numpy().flatten())
aaa = np.vstack(aaa)
#print(aaa.tolist()[0])
pd = pd.DataFrame(aaa.tolist()[0])
pd.to_csv('Chol_predg_1111.csv')

In [32]:
import pandas as pd
bbb = []
bbb.append(target.detach().cpu().numpy().flatten())
bbb = np.vstack(bbb)
# print(aaa.tolist()[0])
pd1 = pd.DataFrame(bbb.tolist()[0])
pd1.to_csv('Chol_targetg_1111.csv')